In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import matplotlib.pyplot as plt

from dftorch.SCF import SCFx, SCFx_batch
from dftorch.Constants import Constants
from dftorch.Structure import Structure, StructureBatch
from dftorch.MD import MDXL, MDXLBatch
from dftorch.ESDriver import ESDriver, ESDriverBatch

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.

torch.cuda.empty_cache()

In [2]:
#torch._functorch.config.donated_buffer = False

In [3]:
%%time
dftorch_params = {
    'coul_method': '!PME',
    'Coulomb_acc': 1e-5, # coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 10.0, # coulomb cutoff
    'PME_order': 4,

    'SCF_MAX_ITER': 20,
    'SCF_TOL': 1e-5,
    'SCF_ALPHA': 0.4, # Linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,
    'KRYLOV_TOL': 1e-6,
    'KRYLOV_TOL_MD': 1e-4,
    'KRYLOV_START': 30,
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'

# filename = ['water_30.xyz']*1
# LBox = torch.tensor([[30,30,30]]*1, device=device)

# filename = ['COORD_8WATER.xyz']*2
# LBox = None
#LBox = torch.tensor([[20,20,20], [9,9,9]], device=device)


# filename = ['C840.xyz']*3
# LBox = torch.tensor([[33,33,33], [37,37,37], [37,37,37]], device=device)

# filename = ['C840.xyz']*2
# LBox = torch.tensor([[34,34,34]]*2, device=device)

# filename = ['COORD.xyz']*2
# LBox = None
# LBox = torch.tensor([[10,15,15], [9,3,3]], device=device)

filename = ['COORD.xyz', 'COORD1.xyz']
LBox = None

# filename = ['C1.xyz', 'C2.xyz']
# LBox = None
# # LBox = torch.tensor([[30,30,30], [15,30,30]], device=device)

# filename = ['C1.xyz']
# LBox = torch.tensor([[30,30,30]], device=device)

# filename = ['C1.xyz']*4
# LBox = torch.tensor([[20,35,35]]*4, device=device)

const = Constants(
    filename,
    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1',
    param_grad = True).to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!
structure = StructureBatch(
    filename, LBox, const, charge=0, e_field=Efield, device=device,
    req_grad_xyz=True)
structure.Te = 3000.0 # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

es_driver = ESDriverBatch(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure, const, do_scf=True, verbose=True)

es_driver.calc_forces(structure, const)

# grads = torch.autograd.grad(structure.f_tot.abs().sum(), const.Es,
#                                 create_graph=False, retain_graph=False, allow_unused=False)
# grads

/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/C-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-C.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-O.skf
  t <batched neighbor list> 0.081 s  (B=2, N=20)
H0_and_S
  Do H off-diag
  t <dR and pair mask> 0.0 s

  t <SKF> 0.0 s

  Do H and S
H0_and_S t 0.1 s

  t <batched neighbor list> 0.003 s  (B=2, N=20)
  t <batched neighbor list> 0.002 s  (B=2, N=20)

Starting cycle
Iter 1
Batch 0: Res = 1.892e+00, dEc = 9.908e+00
Batch 1: Res = 2.36

In [ ]:
(tensor([  0.0000, -19.3348,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000, -27.6949,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000], device='cuda:0'),)


tensor(896.0317, device='cuda:0', grad_fn=<SumBackward0>)

In [26]:
structure.Znuc

tensor([[6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1],
        [6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1, 6, 1, 1]],
       device='cuda:0')

In [ ]:
@torch._dynamo.disable
def force_scalar_from_Es(structure, const, dftorch_params):

    es_driver = ESDriverBatch(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
    es_driver(structure, const, do_scf=True, verbose=True)

    dRX, dRY, dRZ = torch.autograd.grad(structure.e_tot.sum(), (structure.RX, structure.RY, structure.RZ),
                                    create_graph=True, retain_graph=True, allow_unused=False)
    
    forces = -torch.stack((dRX, dRY, dRZ), dim=-2)

    grads = torch.autograd.grad(forces.sum(), const.Es,
                                create_graph=False, retain_graph=False, allow_unused=False)


    return grads

grads = force_scalar_from_Es(structure, const, dftorch_params)


In [4]:

grads



(tensor([0., nan, 0., 0., 0., 0., nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        device='cuda:0'),)

RuntimeError: torch.compile with aot_autograd does not currently support double backward

RuntimeError: torch.compile with aot_autograd does not currently support double backward

In [ ]:
grads = torch.autograd.grad(structure.e_tot.sum(), const.Es,
                                    create_graph=False, retain_graph=True, allow_unused=False)
grads

RuntimeError: One of the differentiated Tensors does not require grad

In [21]:
dF_dEs = torch.autograd.grad(
    grads.sum(),
    const.Es,
    create_graph=False,         # first-order w.r.t. Es
    retain_graph=False,
    allow_unused=False,
)[0]

AttributeError: 'tuple' object has no attribute 'sum'

In [4]:
structure.q

tensor([[ 8.9510e-02, -1.3694e-02, -4.5276e-02, -2.3088e-02, -1.9502e-02,
          1.0423e-01, -5.3912e-05, -2.7984e-02, -4.6090e-03, -7.4986e-02,
          1.0253e-01,  5.8964e-04, -2.9068e-02, -4.2750e-03, -7.3013e-02,
          1.0066e-01, -7.6866e-03,  4.4533e-03,  3.1310e-03, -8.1863e-02],
        [-1.3748e-02,  8.9465e-02, -4.5215e-02, -2.3060e-02, -1.9555e-02,
          1.0550e-01,  3.7628e-04, -3.1054e-02, -5.6714e-03, -7.4756e-02,
          1.0063e-01, -7.6363e-03,  3.7735e-03,  2.4689e-03, -8.1517e-02,
          8.4732e-02, -2.1064e-02, -1.6725e-02, -1.7108e-02, -2.9835e-02]],
       device='cuda:0', grad_fn=<ScatterAddBackward0>)

In [13]:
dRX, dRY, dRZ = torch.autograd.grad(structure.e_tot.sum(), (structure.RX, structure.RY, structure.RZ),
                                    create_graph=False, retain_graph=False, allow_unused=False)
forces = -torch.stack((dRX, dRY, dRZ), dim=-2)

In [ ]:
structure.

In [14]:
forces

tensor([[[-6.9869e+00, -3.7025e-01, -7.1729e+00, -3.8648e+00, -1.2782e+00,
           1.4732e+00,  1.1025e+00, -6.6965e+00, -3.1520e+00,  7.4350e+00,
           1.1924e+00,  1.1820e+00, -6.7688e+00, -3.0947e+00,  7.3146e+00,
           1.0445e+01,  1.8659e+00, -1.0265e+00,  6.6630e-01,  7.7349e+00],
         [-3.0852e-01, -7.4628e-02,  2.3521e-01, -1.2474e+00,  9.3322e-01,
          -7.7198e-03, -2.9620e-01,  3.9792e-01, -1.8921e+00,  1.7789e+00,
           2.4967e-02, -3.4336e-01,  2.9851e-01, -1.7893e+00,  1.8415e+00,
           8.8167e-02, -1.7387e-02, -9.5862e-01, -4.8580e-01,  1.8226e+00],
         [-2.8888e+00, -1.0398e-01, -1.2291e+00, -4.3901e-01,  7.0526e-01,
          -5.2757e-01, -1.7578e-01, -1.6484e+00, -4.5501e-01,  2.7774e+00,
          -5.7136e-01, -2.3689e-01, -1.5548e+00, -4.8203e-01,  2.8784e+00,
           1.0762e+00,  5.4525e-02,  4.0891e-01, -2.5529e-01,  2.6673e+00]],

        [[-3.7003e-01, -6.9899e+00, -7.1754e+00, -3.8713e+00, -1.2783e+00,
           1.3339e+0

In [16]:
(structure.f_tot - forces).abs().max()

tensor(1.1665e-06, device='cuda:0')

In [9]:
structure.f_tot

tensor([[[ 2.7248,  0.2214, -1.7922, -0.2640, -0.8900],
         [-0.1348,  0.1252, -1.1124,  0.0807,  1.0412],
         [-1.1840,  0.0360,  0.7393, -0.3136,  0.7223]],

        [[ 0.2882,  0.8016, -1.0899, -3.3884,  3.3884],
         [ 0.2828,  0.8870, -1.1697,  2.5973, -2.5973],
         [-0.5782,  0.6869, -0.1088,  0.0715, -0.0715]]], device='cuda:0')

In [8]:
shift = [0]
shifts = torch.tensor([[i, j, k] for i in shift for j in shift for k in shift],)
shifts

tensor([[0, 0, 0]])

In [ ]:
torch.manual_seed(0)
temperature_K = torch.tensor([20.0, 30.0], device=structure.device)
mdDriver = MDXLBatch(es_driver, const, temperature_K=temperature_K)
mdDriver.run(structure, dftorch_params, num_steps=60, dt=0.3)

########## Step = 0 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
H0: 1.117 s
H1: 0.289 s
  rank: 0, batch 0, Fel = 0.000243
  rank: 0, batch 1, Fel = 0.000491
  Not converged: 2
  rank: 1, batch 0, Fel = 0.000026
  rank: 1, batch 1, Fel = 0.000065
  Not converged: 0
KER: 0.743 s
F AND E: 2.397 s
ETOT = -329.71785481, EPOT = -329.95321454, EKIN = 0.04653360, T = 17.99997718, ResErr = 0.000000, t = 4.5 s
ETOT = -320.25571938, EPOT = -320.49091422, EKIN = 0.06980039, T = 26.99996577, ResErr = 0.000000, t = 4.5 s
0.01159424 GB


########## Step = 1 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
H0: 1.023 s
H1: 0.003 s
  rank: 0, batch 0, Fel = 0.000300
  rank: 0, batch 1, Fel = 0.000434
  Not converged: 2
  rank: 1, batch 0, Fel = 0.000041
  rank: 1, batch 1, Fel = 0.000091
  Not converged: 0
KER: 0.006 s
F AND E: 0.001 s
ETOT = -329.71911965, EPOT = -330.45714513, EKIN = 0.23409489, T = 90.55183762, ResErr = 0.001657, t = 1.0 s
ETOT = -320

W1204 08:27:07.871458 99771 site-packages/torch/_dynamo/convert_frame.py:964] [2/8] torch._dynamo hit config.recompile_limit (8)
W1204 08:27:07.871458 99771 site-packages/torch/_dynamo/convert_frame.py:964] [2/8]    function: 'torch_dynamo_resume_in_vectorized_nearestneighborlist_batch_at_327' (/home/maxim/Projects/DFTB/DFTorch/src/dftorch/nearestneighborlist.py:327)
W1204 08:27:07.871458 99771 site-packages/torch/_dynamo/convert_frame.py:964] [2/8]    last reason: 2/3: max_K == 7496                                          
W1204 08:27:07.871458 99771 site-packages/torch/_dynamo/convert_frame.py:964] [2/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W1204 08:27:07.871458 99771 site-packages/torch/_dynamo/convert_frame.py:964] [2/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.


  rank: 1, batch 0, Fel = 0.000098
  rank: 1, batch 1, Fel = 0.000019
  Not converged: 0
KER: 0.233 s
F AND E: 0.002 s
ETOT = -329.73194528, EPOT = -333.23671199, EKIN = 2.44806583, T = 946.95301732, ResErr = 0.000910, t = 0.5 s
ETOT = -320.27178626, EPOT = -323.95886107, EKIN = 2.56177833, T = 990.93892529, ResErr = 0.000863, t = 0.5 s
0.011596288 GB


########## Step = 5 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.int32)
H0: 0.257 s
H1: 0.004 s
  rank: 0, batch 0, Fel = 0.000078
  rank: 0, batch 1, Fel = 0.000113
  Not converged: 1
  rank: 1, batch 0, Fel = 0.000078
  rank: 1, batch 1, Fel = 0.000025
  Not converged: 0
KER: 0.006 s
F AND E: 0.001 s
ETOT = -329.73677905, EPOT = -334.30872252, EKIN = 3.49993294, T = 1353.83290000, ResErr = 0.001076, t = 0.3 s
ETOT = -320.27978427, EPOT = -325.08123555, EKIN = 3.67907680, T = 1423.12875484, ResErr = 0.001262, t = 0.3 s
0.0115968 GB


########## Step = 6 ##########
   LMAX: tensor([3, 3], device='cuda:0', dtype=torch.

In [ ]:
%%time
dftorch_params = {
    'coul_method': '!PME',
    'Coulomb_acc': 1e-5, # coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 10.0, # coulomb cutoff
    'PME_order': 4,

    'SCF_MAX_ITER': 20,
    'SCF_TOL': 1e-6,
    'SCF_ALPHA': 0.3, # Linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 30,
    'KRYLOV_TOL': 1e-6,
    'KRYLOV_TOL_MD': 1e-4,
    'KRYLOV_START': 3,
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda'

# filename = 'C840.xyz'
# LBox = torch.tensor([33,33,33], device=device)

# filename = 'COORD.xyz'
# LBox = torch.tensor([9,3,3], device=device)

filename = 'COORD_8WATER.xyz'
LBox = torch.tensor([20,20,20], device=device)

# filename = 'water_30.xyz'
# LBox = torch.tensor([30,30,30], device=device)

# filename = 'COORD.xyz'
# LBox = torch.tensor([9,4,4], device=device)

# filename = 'C1.xyz'
# LBox = torch.tensor([30,30,30], device=device)

# filename = 'C2.xyz'
# LBox = torch.tensor([15,30,30], device=device)

const = Constants(
    filename,
    '/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
    #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/mio-1-1/mio-1-1',
    param_grad = False).to(device)
Efield = 0*0.3*torch.tensor([-.3,0.4,0.0], device=device).T # In arbitrary direction  Works ony in 0-field!!!

structure1 = Structure(
    filename, LBox, const, charge=0, e_field=Efield, device=device,
    req_grad_xyz=False)
structure1.Te = 3000.0 # Some electronic temperature in Kelvin, Possible bug at high tempertures!!!

es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=True)
es_driver.calc_forces(structure1, const)

structure1.e_tot - structure1.e_repulsion

/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/H-O.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-H.skf
/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set/O-O.skf
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

### Do SCF ###
  Initial DM_Fermi

Starting cycle
Iter 1
Res = 1.473967917, dEc = 7.726816588, t = 0.0 s

Iter 2
Res = 0.779885694, dEc = 1.463443100, t = 0.0 s

Iter 3
Res = 0.415319004, dEc = 0.716727998, t = 0.0 s

Iter 4
  rank: 0, Fel = 0.002371
  rank: 1, Fel = 0.000263
  rank: 2, Fel = 0.000023
  rank: 3, Fel = 0.000002
  rank: 4, Fel = 0.000000
Res = 0.222174972, dEc = 0.773845792, t = 0.0 s

Iter 5
  rank: 0, Fel = 0.000003
  rank: 1, Fel = 0.000000
Res = 0.000261210, dEc = 0.000855060, t = 0.0 s

Iter 6
  rank: 0, Fel = 0.000000
Res = 0.000000773, dEc = 0.000000267, t = 0.0 s

CPU times: user 3.83 s, sys: 6

In [7]:
torch.manual_seed(0)
temperature_K = torch.tensor(300.0, device=structure1.device)
mdDriver = MDXL(es_driver, const, temperature_K=temperature_K)
mdDriver.run(structure1, dftorch_params, num_steps=40, dt=0.3)

########## Step = 0 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.434 s
H1: 0.002 s
  rank: 0, Fel = 0.000447
  rank: 1, Fel = 0.000034
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.94996648, EPOT = -873.95685062, EKIN = 0.85311593, T = 274.99965138, ResErr = 0.000000, t = 0.4 s
0.020121088 GB


########## Step = 1 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.416 s
H1: 0.003 s
  rank: 0, Fel = 0.000250
  rank: 1, Fel = 0.000027
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.95456956, EPOT = -874.27317422, EKIN = 1.00228106, T = 323.08263687, ResErr = 0.002024, t = 0.4 s
0.0201216 GB


########## Step = 2 ##########
CoulombMatrix_vectorized
  Coulomb_Real t 0.0 s
   LMAX: 7
  Coulomb_k t 0.4 s

H0: 0.425 s
H1: 0.003 s
  rank: 0, Fel = 0.000317
  rank: 1, Fel = 0.000035
KER: 0.003 s
F AND E: 0.002 s
ETOT = -872.96188839, EPOT = -874.63371253, EKIN = 1.31128582, T = 422.68950046, ResErr = 0.00132